In [ ]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=c62bfc84675f6af40a7c7079c3205bc701758742c9cd46c17dde60b6e247aec7
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/Data Mining/Assignment 3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Data Mining/Assignment 3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [ ]:
TrainCols = list(train_data.columns.values)
TestCols = list(test_data.columns.values)
print(TrainCols)
print(TestCols)

['CoverageField11A', 'CoverageField11B', 'CoverageField1A', 'CoverageField1B', 'CoverageField2A', 'CoverageField2B', 'CoverageField3A', 'CoverageField3B', 'CoverageField4A', 'CoverageField4B', 'CoverageField5A', 'CoverageField5B', 'CoverageField6A', 'CoverageField6B', 'Field10', 'Field11', 'Field7', 'Field8', 'Field9', 'GeographicField10A', 'GeographicField10B', 'GeographicField11A', 'GeographicField11B', 'GeographicField12A', 'GeographicField12B', 'GeographicField13A', 'GeographicField13B', 'GeographicField14A', 'GeographicField14B', 'GeographicField15A', 'GeographicField15B', 'GeographicField16A', 'GeographicField16B', 'GeographicField17A', 'GeographicField17B', 'GeographicField18A', 'GeographicField18B', 'GeographicField19A', 'GeographicField19B', 'GeographicField1A', 'GeographicField1B', 'GeographicField20A', 'GeographicField20B', 'GeographicField21A', 'GeographicField21B', 'GeographicField22A', 'GeographicField22B', 'GeographicField23A', 'GeographicField23B', 'GeographicField24A',

In [ ]:
# Seperate Target column from Train Data
Xtrain = train_data[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = train_data[['QuoteConversion_Flag']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = test_data.copy()
print("Test Set shape:")
print(Xtest.shape)

Train Set shape:
(65000, 595)
(65000, 1)
Test Set shape:
(173836, 596)


In [ ]:
print(Xtest.columns)

Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'GeographicField64'],
      dtype='object', length=596)


In [ ]:
Xtest.drop(["GeographicField64"],axis=1,inplace=True)


In [ ]:
sum(pd.isna(Ytrain).sum())

0

Decision Tree

In [ ]:
#CONSTRUCT DEFAULT DECISION TREE ==================
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
clf_predict=clf.predict(Xtest)


In [ ]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=2)
clf_random.fit(Xtrain, Ytrain)
grid_parm=clf_random.best_params_
print("Best parameters:")
print(grid_parm)



Best parameters:
{'min_samples_split': 80, 'max_depth': 7}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(Xtrain,Ytrain)
clf_predict = clf.predict(Xtest)

In [ ]:
#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, Xtrain, Ytrain, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

=== All AUC Scores ===
[0.94025788 0.93388486 0.92640534 0.93591612 0.9308617  0.93817566
 0.93840354 0.92846993 0.93797917 0.92669959]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9337053783041561


Random Forest

In [ ]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(Xtrain, Ytrain)
rfc_predict=rfc.predict(Xtest)

In [ ]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=2)
rfc_random.fit(Xtrain, Ytrain)
grid_parm_rfc=rfc_random.best_params_
print("Best parameters:")
print(grid_parm_rfc)


Best parameters:
{'n_estimators': 90, 'min_samples_split': 40, 'max_depth': 5}


In [ ]:
#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(Xtrain,Ytrain)
rfc_predict = rfc.predict(Xtest)

In [ ]:
#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(Xtrain,Ytrain)
rfc_predict = rfc.predict(Xtest)

KNN

In [ ]:
#KNN =============================================================
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(Xtrain, Ytrain)
knn_prediction=knn.predict(Xtest)

MLP

In [ ]:
#Construct Multilayer Perceptron using the best parameters
#Default mode
mlp = MLPClassifier(random_state=1, max_iter=3)
mlp.fit(Xtrain, Ytrain)
mlp_predict=mlp.predict(Xtest)

Support Vector Machines

In [ ]:
#attempt 1 textbook
svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])
svm_clf.fit(Xtrain, Ytrain)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [ ]:
SMOTE

imblearn.over_sampling._smote.SMOTE

In [ ]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Ytrain))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(Xtrain, Ytrain)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({'QuoteConversion_Flag': 1})
Resampled dataset shape Counter({0: 52738, 1: 26369})


Stacking

In [ ]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using andomForest and Decision Tree Classifier, MLP, KNN, SVC\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(), KNeighborsClassifier(), LinearSVC() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, Xtest,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=2, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using andomForest and Decision Tree Classifier, MLP, KNN, SVC

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.89616727]
    fold  1:  [0.89962835]
    ----
    MEAN:     [0.89789781] + [0.00173054]
    FULL:     [0.89789778]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92172726]
    fold  1:  [0.92106793]
    ----
    MEAN:     [0.92139760] + [0.00032966]
    FULL:     [0.92139760]

model  2:     [MLPClassifier]
    fold  0:  [0.84876372]
    fold  1:  [0.85917630]
    ----
    MEAN:     [0.85397001] + [0.00520629]
    FULL:     [0.85396994]

model  3:     [KNeighborsClassifier]
    fold  0:  [0.67616423]
    fold  1:  [0.67388062]
    ----
    MEAN:     [0.67502242] + [0.00114181]
    FULL:     [0.67502244]

model  4:  

In [ ]:
#STACKING - CONTRUCT A Decision Tree MODEL==============================
stackedmodel = DecisionTreeClassifier()
    
stackedmodel = stackedmodel.fit(S_Train, y_res)
y_pred1 = stackedmodel.predict(S_Test)

In [ ]:
#Hyperparameter tuning done for decision tree classifier
parameters1={'min_samples_split' : range(10,100,10),'max_depth': range(1,10,2)}
clf_random1 = RandomizedSearchCV(stackedmodel,parameters1,n_iter=2)
clf_random1.fit(X_res, y_res)
grid_parm1=clf_random1.best_params_
print("Best parameters:")
print(grid_parm1)

Best parameters:
{'min_samples_split': 90, 'max_depth': 9}


Export for Kaggle

In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability1 = pd.DataFrame(stackedmodel.predict_proba(S_Test))

pred_Probability1.head()

,0,1
0,0.973409,0.026591
1,0.973409,0.026591
2,0.950126,0.049874
3,0.973409,0.026591
4,0.973409,0.026591


In [ ]:
y_pred1 = pd.Series(y_pred1)

In [ ]:
print(y_pred1)

0         0
1         0
2         0
3         0
4         0
         ..
173831    0
173832    0
173833    1
173834    0
173835    0
Length: 173836, dtype: int64


In [ ]:
y_pred1.to_csv('/gdrive/My Drive/Data Mining/Assignment 3/SMOTESackedDTresults2.csv')